In [1]:
import torch
import os
from tqdm import tqdm

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch import nn, optim
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
from PIL import Image
from time import time
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('data/training/labels-tab-csv.csv')
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
image_arr = np.asarray(train.iloc[:, 0])
label_arr = np.asarray(train.iloc[:, 1])

train_img = []
for img_name in tqdm(image_arr):
    # defining the image path
    image_path = 'data/training/' + str(img_name)
    # reading the image
    img = imread(image_path, as_gray=True)
    
  #  t = transforms.Resize(60, 171)
    img = resize(img, (56, 56),
                       anti_aliasing=True)

    # normalizing the pixel values
 #   img /= 255.0
    img = np.true_divide(img, 255) #avoid division error
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = label_arr


train_x.shape


100%|███████████████████████████████████████████████████████████████████████████████| 859/859 [00:02<00:00, 314.31it/s]


(859, 56, 56)

In [3]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((773, 56, 56), (773,)), ((86, 56, 56), (86,)))

In [4]:
train_x = train_x.reshape(773, 1, 56, 56)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format

train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)


train_y = train_y.to(torch.long)
# shape of training data
train_x.shape, train_y.shape
print(train_x[0].dtype)
print(train_y[0].dtype)

torch.float32
torch.int64


In [5]:
val_x = val_x.reshape(86, 1, 56, 56)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

val_y = val_y.to(torch.long)
# shape of validation data
val_x.shape, val_y.shape
print(val_x.dtype)
print(val_y.dtype)

torch.float32
torch.int64


In [6]:
#cnn model class
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(196, 11) #changed from 196 to 2520
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [7]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=11, bias=True)
  )
)


In [8]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)
    

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [9]:

# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)




Epoch :  1 	 loss : tensor(2.5373, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  3 	 loss : tensor(7.4051, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  5 	 loss : tensor(3.6430, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  7 	 loss : tensor(2.9601, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  9 	 loss : tensor(2.4168, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  11 	 loss : tensor(2.4470, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  13 	 loss : tensor(2.4097, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  15 	 loss : tensor(2.4013, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  17 	 loss : tensor(2.3887, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  19 	 loss : tensor(2.3759, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  21 	 loss : tensor(2.3642, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  23 	 loss : tensor(2.3587, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  25 	 loss : tensor(2.355

In [10]:
with torch.no_grad():
    output = model(train_x.cuda())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

0.11254851228978008

In [11]:
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)


0.08139534883720931